In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor

In [ ]:
def normalize(dataset):
    mean, std = dataset.mean(), dataset.std()
    result = (dataset - mean)/std

    print("min, max", result.min(), result.max())
    print("mean, std", mean, std)
    return result

dir_path = "dataset/1e8_unique"
dataset_np = np.load("%s/kolakosky.npy" % dir_path).astype(np.float32)
idxs_train = np.load("%s/train_idxs.npy" % dir_path)
idxs_valid = np.load("%s/valid_idxs.npy" % dir_path)

# normalize numbers
dataset_np[:, 1:-1] = normalize(dataset_np[:, 1:-1])

print(dataset_np.shape)

In [ ]:
dataset_np[0, 1:-1], dataset_np[:, 1:-1].min(), dataset_np[:, 1:-1].max()

In [ ]:
# datasets
trainset, validset = torch.Tensor(dataset_np[idxs_train]), torch.Tensor(dataset_np[idxs_valid])

In [ ]:
# dataloader
batch_size = 1500
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=5)
validloader = torch.utils.data.DataLoader(validset, batch_size=len(validset), shuffle=False, num_workers=1)

print("trainset", len(trainset), len(trainloader))
print("validset", len(validset), len(validloader))

In [ ]:
def eval_epoch(net, dloader, epoch):
    with torch.no_grad():
        valid_loss = 0
        valid_acc = 0
        num_correct = 0
        
        for i, data in enumerate(dloader):
            inputs, labels = data[:, 1:-1].cuda(), data[:, -1].long().cuda()
            outputs = net(inputs)

            # loss
            valid_loss += criterion(outputs, labels).item() * data.shape[0]

            # acc
            (_, arg_maxs) = Tensor.max(outputs.data, dim=1)
            num_correct += Tensor.sum(labels==arg_maxs)
    
    acc = (num_correct * 100.0 / len(validset)).item()
    loss = valid_loss/len(validset)
    print("[Epoch %d] Eval Acc %.2f%% (Loss %.3f)" % (epoch, acc, loss))
    return acc, loss

In [ ]:
class Net1(nn.Module):
    def __init__(self, n_inputs):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(n_inputs, 34)
        self.fc2 = nn.Linear(34, 50)
        self.fc3 = nn.Linear(50, 256)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Net2(nn.Module):
    def __init__(self, n_inputs):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(n_inputs, 60)
        self.fc2 = nn.Linear(60, 150)
        self.fc3 = nn.Linear(150, 250)
        self.fc4 = nn.Linear(250, 300)
        self.fc5 = nn.Linear(300, 500)
        self.fc6 = nn.Linear(500, 256)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x
    
class Net3(nn.Module):
    def __init__(self, n_inputs):
        super(Net3, self).__init__()
        self.fc1 = nn.Linear(n_inputs, 60)
        self.fc2 = nn.Linear(60, 150)
        self.fc3 = nn.Linear(150, 250)
        self.fc4 = nn.Linear(250, 300)
        self.fc5 = nn.Linear(300, 500)
        self.fc6 = nn.Linear(500, 700)
        self.fc7 = nn.Linear(700, 500)
        self.fc8 = nn.Linear(500, 300)
        self.fc9 = nn.Linear(300, 256)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = self.fc9(x)
        return x

net = Net3(n_inputs=40).cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.AdamW(net.parameters(), lr=1e-3)
last_saved_eval_acc = 90

In [ ]:
total_params = sum(p.numel() for p in net.parameters())
print("Total params", total_params)

In [ ]:
# optimizer = optim.AdamW(net.parameters(), lr=1e-4)

In [ ]:
print_iter = int(150000/batch_size)

class_str = str(net.__class__).replace("'>", "").split(".")[1]
# CHECKPOINT_PATH = "nn_checkpoints/40feat_unique_81e8_FConly_%.0f_%s.pth" % (last_saved_eval_acc, class_str)
# net.load_state_dict(torch.load(CHECKPOINT_PATH))

for epoch in range(1, 500):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[:, 1:-1].cuda(), data[:, -1].long().cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % print_iter == print_iter-1:    # print every 20 mini-batches
            print('[%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, running_loss / print_iter))
            running_loss = 0.0
    eval_acc, _ = eval_epoch(net, validloader, epoch)
    if int(eval_acc) > last_saved_eval_acc:
        last_saved_eval_acc = int(eval_acc)
        CHECKPOINT_PATH = "nn_checkpoints/40feat_unique_81e8_FConly_%.0f_%s.pth" % (last_saved_eval_acc, class_str)
        torch.save(net.state_dict(), CHECKPOINT_PATH)

print('Finished Training')